In [1]:
edge_driver_path = 'C:/Users/Hadi Khaled/Desktop/msedgedriver.exe'
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [ ]:
# extract links 

driver = webdriver.Edge(executable_path=r'C:/Users/msedgedriver.exe')

# Empty lists to store links
categorical_links = []
personal_links = []
newmain =['//op.europa.eu/en/web/who-is-who/organization/-/organization/SJ/SJ']

try:
    for item in newmain:
        try:
            indexs = newmain.index(item)
            # Navigate to the URL
            driver.get('https:' + item)

            # Locate and click the "Sublevels" tab
            sublevels_tab = driver.find_element_by_id('_eu_europa_publications_portlet_wiw_OrganizationDetailPortlet_SublevelsBtn')
            sublevels_tab.click()

            # Wait for the page to load completely
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//li[@class="tree-node"]')))

            # Get the page source after waiting for the dynamic content to load
            page_source = driver.page_source

            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(page_source, 'html.parser')

            # Find categorical links
            new_c_links = [a['href'] for a in soup.select('span.tree-label > a')]
            if new_c_links:
                newmain[indexs + 1:indexs + 1] = new_c_links

            # Find personal links
            new_p_links = [a['href'] for a in soup.select('.wiw-sublevel-person-name > a')]
            if new_p_links:
                personal_links.extend(new_p_links)
        except Exception as e:
            print(f"Error: {e}")
finally:
    # Close the webdriver
    driver.quit()

# Print the lists of links
print("Categorical Links:")
print(categorical_links)
print("\nPersonal Links:")
print(personal_links)


In [ ]:
#####extract data from links 

# Create a session object to maintain a persistent connection to the website
session = requests.Session()

data = []
failed_links = []  # List to store links that could not be scraped
counter = 0

for url in project_links[2050:2177]:
    
    attempt = 0
    success = False
    project_data = None
    
    while attempt < 10 and not success:
        try:
            # Send a GET request to the website and get the HTML content
            URL_HOME = 'https:'
            response = session.get(URL_HOME + url, timeout=None)
            
            while True:
                # Check if the response status code is 200
                if response.status_code == 200:
                    counter += 1
                    print("Counter:", counter)
                    # Parse the HTML content using the lxml parser
                    soup = BeautifulSoup(response.content, 'lxml')
                    break
                else:
                    response.close()
                    response = session.get(URL_HOME + url, timeout=None)
                    print(f"Failed to fetch URL: {url}, Status Code: {response.status_code}")    
                    continue

            # Find individual elements within the parent div
            name_element = soup.find('h1', id='maincontentgo')
            title_element = soup.find('span', class_='wiw-person-detail-current-position')
            phone_element = soup.find('div', id='_eu_europa_publications_portlet_wiw_PersonDetailPortlet__phone')
            # Find the div with id "firstPosition"
            first_position_div = soup.find('div', id='firstPosition')

            # Check if the div exists
            if first_position_div:
                # Find all <a> tags within the div
                a_tags = first_position_div.find_all('a')
                # Extract the text from the last <a> tag
                level = a_tags[-1].text.strip() if a_tags else None
            else:
                level = None

            # Extract the text from other elements
            name = name_element.text.strip() if name_element else None
            title = title_element.text.strip() if title_element else None
            phone = phone_element.text.strip() if phone_element else None

            # Create a dictionary with the data
            project_data = {
                'Name': name,
                'Title': title,
                'Phone': phone,
                'Level': level
            }
            
            success = True  # Scraping succeeded
        except Exception as e:
            attempt += 1
            if attempt == 10:
                # If all retry attempts failed, add the URL to the list of failed links
                failed_links.append(url)
                print(f"Failed to scrape URL: {url}, Error: {str(e)}")

    # Append the dictionary to the list if scraping was successful
    if success:
        data.append(project_data)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)



output_file = 'E:/Projects/Project.xlsx'



# Read the existing data in Sheet1 if it exists
existing_data = pd.DataFrame()
try:
    existing_data = pd.read_excel(output_file, sheet_name='Sheet1')
except FileNotFoundError:
    pass  # No existing data, so leave existing_data as an empty DataFrame

# Concatenate the existing data with the new data
combined_data = pd.concat([existing_data, df], ignore_index=True)

# Write the combined data to the Excel file starting from the last row of existing data
with pd.ExcelWriter(output_file, mode='w') as writer:
    combined_data.to_excel(writer, sheet_name='Sheet1', index=False)

# Print the links that could not be scraped
print("Failed links:")
for link in failed_links:
    print(link)


In [ ]:
#extract emails

# Initialize Edge webdriver
driver = webdriver.Edge(executable_path=r'C:/Users/msedgedriver.exe')

data = []

try:
    for item in project_links[101:500]:
        try:
            # Navigate to the URL
            driver.get('https:' + item)

            span_element = WebDriverWait(driver, 1000).until(EC.element_to_be_clickable((By.CLASS_NAME, 'address-email')))

            # Click on the span element
            span_element.click()

            # Wait for the page to load completely
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//span[@class="address-email"]')))

            # Get the page source after waiting for the dynamic content to load
            page_source = driver.page_source

            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(page_source, 'html.parser')

            # Find the anchor tag inside the span element
            email_link = soup.find('span', class_='address-email').find('a')

            # Extract the email address and the mailto link
            email_address = email_link.text
            
            # Create a dictionary with the data
            project_data = {
                'email_address': email_address
            }

            print("Email address:", email_address)

            data.append(project_data)
            
        except Exception as e:
            print(f"Error: {e}")

finally:
    # Close the webdriver
    driver.quit()

# Specify the path for the Excel file
email_file = 'E:/Projects/email.xlsx'

# Check if the Excel file exists
if not os.path.exists(email_file):
    # If the file doesn't exist, write the DataFrame to a new Excel file
    with pd.ExcelWriter(email_file, mode='w') as writer:
        pd.DataFrame(data).to_excel(writer, sheet_name='Sheet1', index=False)
else:
    # Read the existing data in Sheet1 if it exists
    existing_data = pd.DataFrame()
    try:
        existing_data = pd.read_excel(email_file, sheet_name='Sheet1')
    except FileNotFoundError:
        pass  # No existing data, so leave existing_data as an empty DataFrame

    # Concatenate the existing data with the new data
    combined_data = pd.concat([existing_data, pd.DataFrame(data)], ignore_index=True)

    # Write the combined data to the Excel file starting from the last row of existing data
    with pd.ExcelWriter(email_file, mode='w') as writer:
        combined_data.to_excel(writer, sheet_name='Sheet1', index=False)
